# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [9]:
# Write your code below.

%load_ext dotenv
%dotenv 

import os
import sys
sys.path.append(os.getenv('SRC_DIR'))
from utils.logger import get_logger
_logs = get_logger(__name__)



The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [10]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [11]:
import random

ft_dir = os.getenv('PRICE_DATA')

from glob import glob

ft_glob = glob(os.path.join(ft_dir, '**/*.parquet'), 
               recursive = True)

random.seed(42)
ft_glob = random.sample(ft_glob, 60)



dt_list = []
for s_file in ft_glob:
    _logs.info(f"Reading file: {s_file}")
    dt = dd.read_parquet(s_file)
    dt_list.append(dt)
price_data = dd.concat(dt_list, axis = 0, ignore_index = True)





2025-06-08 19:18:24,905, 660446801.py, 17, INFO, Reading file: ../../05_src/data/prices/NVS/NVS_2015/part.0.parquet
2025-06-08 19:18:24,915, 660446801.py, 17, INFO, Reading file: ../../05_src/data/prices/BLX/BLX_2018/part.0.parquet
2025-06-08 19:18:24,918, 660446801.py, 17, INFO, Reading file: ../../05_src/data/prices/BKTI/BKTI_1997/part.0.parquet
2025-06-08 19:18:24,922, 660446801.py, 17, INFO, Reading file: ../../05_src/data/prices/PNR/PNR_2008/part.1.parquet
2025-06-08 19:18:24,926, 660446801.py, 17, INFO, Reading file: ../../05_src/data/prices/CIK/CIK_2017/part.0.parquet
2025-06-08 19:18:24,930, 660446801.py, 17, INFO, Reading file: ../../05_src/data/prices/RMD/RMD_2008/part.1.parquet
2025-06-08 19:18:24,934, 660446801.py, 17, INFO, Reading file: ../../05_src/data/prices/AMAT/AMAT_1990/part.1.parquet
2025-06-08 19:18:24,937, 660446801.py, 17, INFO, Reading file: ../../05_src/data/prices/CTSO/CTSO_2017/part.1.parquet
2025-06-08 19:18:24,940, 660446801.py, 17, INFO, Reading file: ../

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [12]:
dd_shift = price_data.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1=x['Close'].shift(1),
        Adj_Close_1=x['Adj Close'].shift(1),
        hi_lo_range = lambda x: x['High'] - x['Low']
    ),
  
)

dd_feat = dd_shift.assign(
    returns=lambda df: df['Close'] / df['Close_lag_1'] - 1
)





/var/folders/qk/4stqp5452fzcdbt3_5q8j9xw0000gn/T/ipykernel_47839/1693657292.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = price_data.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [13]:
#convert to Pandas

pandas_df_feat = dd_feat.compute()
pandas_df_feat['returns_ma_10'] = pandas_df_feat.groupby('ticker')['returns'].transform(lambda x: x.rolling(10).mean())
pandas_df_feat

,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_1,hi_lo_range,returns,returns_ma_10
134922,2015-01-02,0.9100,0.910,0.9100,0.9100,0.910000,0.0,ACB.csv,ACB,2015,NaN,NaN,0.0000,NaN,NaN
134923,2015-01-05,0.9100,0.910,0.9100,0.9100,0.910000,0.0,ACB.csv,ACB,2015,0.9100,0.910000,0.0000,0.000000,NaN
134924,2015-01-06,0.9100,0.910,0.9100,0.9100,0.910000,0.0,ACB.csv,ACB,2015,0.9100,0.910000,0.0000,0.000000,NaN
134925,2015-01-07,0.9100,0.910,0.9100,0.9100,0.910000,0.0,ACB.csv,ACB,2015,0.9100,0.910000,0.0000,0.000000,NaN
134926,2015-01-08,0.9100,0.910,0.9100,0.9100,0.910000,0.0,ACB.csv,ACB,2015,0.9100,0.910000,0.0000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197058,1998-06-26,13.1250,13.500,13.1250,13.3750,12.249803,19100.0,ZEUS.csv,ZEUS,1998,13.3750,12.249803,0.3750,0.000000,-0.003610
197059,1998-06-29,13.3750,13.750,13.0000,13.0000,11.906351,85700.0,ZEUS.csv,ZEUS,1998,13.3750,12.249803,0.7500,-0.028037,-0.005512
197060,1998-06-30,13.1250,13.125,12.3750,12.5000,11.448412,92200.0,ZEUS.csv,ZEUS,1998,13.0000,11.906351,0.7500,-0.038462,-0.009359
197061,1998-07-01,12.7500,12.750,12.4375,12.5625,11.505656,69800.0,ZEUS.csv,ZEUS,1998,12.5000,11.448412,0.3125,0.005000,-0.007950


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

> A1. No, converting to Pandas is not strictly necessary, but if  you have small datasets (or if it fits in your memory) is the simplest and safest option.

>A2. It depends of the size of the datasets. For larger datasets or scalable pipelines, staying with Dask would be more appropriate to benefit from its scalability.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.